# XAI Classification of Lateral Spreading

**Exercise:** 
[![Try on DesignSafe](https://raw.githubusercontent.com/DesignSafe-Training/xai/main/DesignSafe-Badge.svg)](https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/Training/xai/01-classification-exercise.ipynb)

**Solution:** [![Try on DesignSafe](https://raw.githubusercontent.com/DesignSafe-Training/xai/main/DesignSafe-Badge.svg)](https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/Training/xai/01-classification.ipynb)

### Install packages

In [ ]:
!pip3 install scikit-learn pandas --quiet
!pip3 install xgboost --quiet
!pip3 install shap --quiet

## Liquefaction

> Durante, M. G., & Rathje, E. M. (2021). An exploration of the use of machine learning to predict lateral spreading. Earthquake Spectra, 37(4), 2288-2314.

Soil liquefaction is a phenomenon that typically occurs in saturated loose sandy soils subjected to rapid loading conditions, such as earthquakes. The generation of excess pore water pressure is a direct consequence of the rapid loading, which can lead to a sudden reduction in the strength and stiffness of the soil. In the presence of gently sloping ground or near the free face of a slope, the occurrence of earthquake-induced liquefaction may generate lateral displacements, known as lateral spreading.

![liquefaction](liquefaction.png)

> Fig: (a) Observed liquefaction-related damage (data from NZGD, 2013), and (b) lateral spreading
horizontal displacement observed from optical image correlation (data from Rathje et al., 2017b) in the
Avon River area for the 2011 Christchurch earthquake

## Lateral spreading classification

Durante and Rathje (2021) classified sites that experienced more than 0.3 m displacement as lateral spreading. We now evaluate different factors that influence soil lateral spreading, such as (i) Ground Water Table (ground water closer to surface means more chance of liquefaction), (ii) slope angle (steeper the slope more lateral spreading), (iii) PGA - Peak Ground acceleration (intensity of earthquake shaking), and (iv) elevation (certain sites on high terrace don't show lateral spreading).

![Factors affecting liquefaction](liq-factors.png)

> Image credits: Durante and Rathje (2021).

### Explore data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Read data from DesignSafe community data

In [ ]:
# Explore your dataset

### Filtering for features

Remove any feature in the dataset that we don't want to include in the training process.

In [ ]:
df = df.drop(['Test ID', 'Elevation'], axis=1)
df.head()

In [ ]:
X = df.copy(deep=True)
y = df['Target']
y.head()

### Training, testing and validation

When developing a machine learning model, it is common practice to divide the available data into three subsets - training, validation, and testing. This is done to properly assess model performance and generalizability.

The training set is used to fit the parameters of the model. The majority of the data, typically 60-80%, is allocated for training so that the model can learn the underlying patterns.

The validation set is used to tune any hyperparameters of the model and make architectural choices. For example, it helps decide the number of hidden layers and units in a neural network. Typically 20% of data is used for validation.

The test set provides an unbiased evaluation of the fully-trained model's performance. It is critical for getting an accurate estimate of how the model will work on new unseen data. Usually 20% of the data is reserved for testing.

The splits should be made randomly while ensuring the class distribution is approximately balanced across all sets. No data from the validation or test sets should be used during training. This prevents overfitting and ensures the evaluation reflects real-world performance.

In [ ]:
# View data before splitting to training and testing

We are going to use the `train_test_split` function twice to split the data into training + validation and testing. Then, we split the `training + validation` into training and validation. We retrain the target values for now, so we can check how good is our prediction.

In [ ]:
# Specify split
X_train_target, X_val_test_target, y_train, y_val_test = train_test_split(X, y, test_size=)
X_test_target, X_val_target, y_test, y_val = train_test_split(X_val_test_target, y_val_test, test_size=)

In [ ]:
X_val_target

In [ ]:
X_train = X_train_target.drop(['Target'], axis=1)
X_test = X_test_target.drop(['Target'], axis=1)
X_val = X_val_target.drop(['Target'], axis=1)

In [ ]:
X_test.head()

In [ ]:
y_test.head()

# Decision tree classifier

## Classification: Decision Trees

Classification is the task of predicting a categorical target variable based on input data. There are two main types of classification:

**Binary classification**: The target variable has two possible classes, often labeled 0 and 1. The goal is to predict which of the two classes an input belongs to. Examples include spam detection, disease diagnosis, etc.

**Multi-class classification**: The target variable has more than two possible discrete values or classes. The goal is to predict the specific class an input belongs to out of the multiple choices. Examples include image recognition, document categorization, etc.

A decision tree is a flowchart-like structure where each internal node represents a test on a feature (e.g., "is feature A > 5?"), each branch represents an outcome of the test, and each leaf node represents a class label. A decision tree is a supervised learning model used for both binary and multi-class classification. It works by recursively partitioning the input space into smaller subspaces based on the value of different predictor variables. The goal is to create leaf nodes that contain cases with similar values of the target variable.

The process of learning a decision tree involves selecting features and split points that best separate the classes, based on a criterion such as information gain or Gini impurity. It continues until a stopping criterion is met, like reaching a maximum depth or a minimum number of samples per leaf.

The structure of a decision tree consists of:

  *  __Root node__: This is the topmost node in the tree and contains the full dataset.
  *  __Internal nodes__: These represent points where the data is split based on values of predictor variables.
  *  __Branches__: These connect the internal nodesbased on possible values of predictors.
  *  __Leaf nodes__: These represent the final classifications or predictions.

Decision trees can handle both categorical and continuous predictors.

Some key advantages of decision trees are:

* __Interpretability__ - The tree structure and rules are easy to understand.
* __Non-parametric__ - No assumptions about data distribution.
* __Handles nonlinear relationships__ - By partitioning data recursively.
* __Handles categorical variables__ - No need for dummy coding.

![liquefaction decision tree](liq-dt.png)

### GINI Impurity

Gini impurity is used to evaluate how good a split is by calculating the impurity of the subsets resulting from the split. A lower Gini score means that the split is separating the classes well. It quantifies the disorder or uncertainty within a set of data.

The Gini impurity for a binary classification is calculated as:
$$Gini(t)=1−\sum(p_i)^2$$

where $p_i$​ is the probability of class $i$ in the set.

### Decision Tree splitting

A decision tree decides a split by selecting a feature and a value to divide the dataset into two or more homogenous subsets, according to a certain criterion. The ultimate goal is to find the splits that produce the purest subsets, meaning that each subset ideally contains data points from only one class. Here's how it works:

  - **Selection of Criteria**: The method used to decide a split depends on the criterion being used. Common criteria for classification tasks include Gini Impurity, Information Gain, and Chi-Squared. For regression tasks, variance reduction is often used.

  - **Evaluate Each Feature and Potential Split**: For each feature in the dataset, the algorithm calculates the criterion's value for every potential split point. Split points can be the actual values of a continuous feature or different categories of a categorical feature.

  - **Choose the Best Split**: The algorithm selects the feature and split point that produces the subsets with the highest purity according to the chosen criterion. For example:
        In the case of Gini Impurity, the best split minimizes the impurity.
        In the case of Information Gain, the best split maximizes the gain.

  - **Create Subsets**: Once the best split has been identified, the dataset is divided into subsets according to the chosen feature and split point.

  - **Repeat**: Steps 1-4 are repeated recursively on each of the subsets until a stopping criterion is met, such as reaching a certain tree depth or the subsets being pure enough.

#### Example Using Gini Impurity

  - For each feature, consider all possible values for splitting.
  - Choose the split that results in the lowest weighted Gini Impurity.
  - Calculate the Gini Impurity for each possible split as:
  - Divide the dataset accordingly and continue the process.

In the context of decision trees, the splitting process is essential as it helps the model generalize the pattern from the training data, enabling accurate predictions or classifications for unseen data. It's the core step in building the tree, and different algorithms might have variations in the splitting procedure.

## Decision tree with SciKit Learn

Scikit-learn, often referred to as sklearn, is one of the most popular libraries for machine learning in Python

In [ ]:
from sklearn import tree
from sklearn.metrics import accuracy_score

In [ ]:
print('Training score: %.2f%%' %(clf.score(X_train, y_train) * 100))
print('Validation score: %.2f%%' %(clf.score(X_val, y_val) * 100))
print('Testing score: %.2f%%' %(clf.score(X_test, y_test) * 100))

In [ ]:
# View accuracy scores

#### Prediction

In [ ]:
# Make a prediction

### Visualizing a decision tree

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 20))
tree.plot_tree(clf,
               feature_names =['GWD (m)', 'L (km)', 'Slope (%)', 'PGA (g)'],
               class_names=['Liquefaction', 'No liquefaction'],
               filled = True)
fig.show()

### Update model

We increase the `max_depth` to 7 so we have a better fit.

> 💡 Try varying the `max_depth` from 5 - 9.

In [ ]:
# Specify depth
clf = tree.DecisionTreeClassifier(max_depth= )
clf.fit(X_train, y_train)

In [ ]:
print('Training score: %.2f%%' %(clf.score(X_train, y_train) * 100))
print('Validation score: %.2f%%' %(clf.score(X_val, y_val) * 100))
print('Testing score: %.2f%%' %(clf.score(X_test, y_test) * 100))

### Summary of decision trees


Pros and Cons
- Pros: Simple to understand and visualize, able to handle categorical and numerical data.
- Cons: Prone to overfitting, especially when the tree is deep, leading to poor generalization to unseen data. A small change in the data can lead to a very different tree.

# XGBoost (Extreme Gradient Boosting)

XGBoost is a popular and efficient gradient boosting framework that's widely used in machine learning, including in scientific contexts. It builds on the idea of boosting, where weak learners are combined to create a strong learner.

## Gradient Boosting Framework

Gradient Boosting is an ensemble learning method that fits a sequence of weak learners (such as shallow decision trees) on modified versions of the data.

The general form of the boosting model is:

$$ f(x) = \sum_{k=1}^K \alpha_k h_k(x) $$

Where:
- $f(x)$ is the prediction for input $ x $
- $ \alpha_k $ is the weight of the $ k $-th weak learner
- $ h_k(x) $ is the prediction of the $ k $-th weak learner
- $ K $ is the total number of weak learners

## Algorithm

1. **Initialization:** Start with a constant prediction $ f_0(x) $
2. **Iteratively Add Trees:** For $ k = 1 $ to $ K $:
   a. Compute the negative gradient (or "pseudo-residuals") of the loss function:
    $$ r_i = -\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} $$
   b. Fit a weak learner to predict the pseudo-residuals.
   c. Compute the weight $ \alpha_k $ that minimizes the loss.
   d. Update the model by adding the weighted weak learner:
      $$ f_k(x) = f_{k-1}(x) + \alpha_k h_k(x) $$

## XGBoost Enhancements

XGBoost adds several enhancements to the basic gradient boosting framework:

### Regularization

XGBoost includes L1 (Lasso) and L2 (Ridge) regularization terms in the loss function to prevent overfitting:

$$ \text{Loss} = L(y, f(x)) + \lambda_1 \sum |\alpha_k| + \lambda_2 \sum \alpha_k^2 $$

### Handling Missing Data

XGBoost can automatically learn the best direction to handle missing values during training, allowing it to handle datasets with missing data.

### Column Block and Parallelization

XGBoost employs a column block structure that allows parallelization over both instances and features, leading to efficient computation.

### Hyperparameters

- **learning_rate:** Step size shrinkage to prevent overfitting.
- **max_depth:** Maximum depth of the decision trees.
- **n_estimators:** Number of boosting rounds.
- **subsample:** Fraction of the data to be used for each boosting round, enabling stochastic gradient boosting.

![XGB](xgb.png)

In [ ]:
# Create and train XGB model

In [ ]:
print('Training score: %.2f%%' %(xgb.score(X_train, y_train) * 100))
print('Validation score: %.2f%%' %(xgb.score(X_val, y_val) * 100))
print('Testing score: %.2f%%' %(xgb.score(X_test, y_test) * 100))

## Explainable AI

### Feature Importance Metrics

Feature importance metrics provide insights into the contribution of individual features (or variables) to a predictive model. Understanding the importance of different features can help in feature selection, model interpretation, and understanding the underlying relationships within the data.

#### Types of Importance Metrics

##### 1. **Weight-Based Importance**

In models like linear regression or logistic regression, the magnitude of the coefficients can indicate the importance of features. For tree-based models like Random Forest, the importance of a feature can be calculated based on the average gain of the feature when it is used in trees.

##### 2. **Permutation Importance**

Permutation importance is a method that can be applied to any model. It is computed as follows:

a. **Calculate a Performance Metric:** Train the model and calculate a performance metric (such as accuracy or MSE) using a validation set.
b. **Permute the Feature Values:** Shuffle the values of the feature of interest across the validation set, destroying the relationship between the feature and the target.
c. **Recompute the Performance Metric:** Use the permuted data to compute the performance metric again.
d. **Calculate Importance:** The difference between the original metric and the permuted metric provides a measure of importance.

In [ ]:
import numpy as np
## Compute feature importance for random forest
def plot_feature_importance(X, model):
    df2=pd.get_dummies(X)
    features = df2.columns
    importances = model.feature_importances_
    indices = np.argsort(importances)[-10:]  # top 10 features
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

    fi = pd.DataFrame({'feature': list(X.columns),
                       'importance': model.feature_importances_}).\
                       sort_values('importance', ascending = False)

    return fi

In [ ]:
# Display results for decision tree
plot_feature_importance(X_test, clf)

In [ ]:
# Display results for XGB
plot_feature_importance(X_test, xgb)

### 3. Shapley Values


SHAP values are based on cooperative game theory and provide a unified measure of feature importance. They give the average contribution of a feature value to every possible prediction.

For a model with $M$ features, the Shapley value for feature $ i $ is computed as:

$$ \phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(M - |S| - 1)!}{M!} [v(S \cup \{i\}) - v(S)] $$

Where:
- $ S $ is a subset of features excluding feature $ i $
- $ N $ is the set of all features
- $ v(S) $ is the value function that gives the prediction for subset $ S $


### Understanding SHAP Values: An Analogy

SHAP values help us understand how different features (or variables) contribute to a prediction in a machine learning model. To explain this without mathematics, let's use an analogy of three team members working together on a project. Imagine that the team members are Alice, Bob, and Charlie, and we want to know how much each one contributed to the project's success.

#### The Project

The project's success is measured by the total output, and we want to fairly distribute the credit for this success among Alice, Bob, and Charlie.

#### Shapley Value Concept

Imagine breaking down the project into smaller tasks and observing how much the output changes when each team member is added or removed. The goal is to find the average contribution of each member over all possible ways the team could have been formed.

##### Step by Step Explanation

1. **No Team Members:** First, we measure the output with no team members working (a baseline).
2. **Adding One Member:** Then, we add each team member one by one and measure how much the output changes.
   - With Alice alone
   - With Bob alone
   - With Charlie alone
3. **Adding Two Members:** Next, we measure the change in output with pairs of team members:
   - Alice and Bob
   - Alice and Charlie
   - Bob and Charlie
4. **All Three Members:** Finally, we measure the output with all three working together.

#### Calculating Contributions

By comparing all these different combinations, we can calculate the average contribution of each team member. The SHAP value for each person is their fair share of the contribution to the project's success.

- **Alice's SHAP Value:** The average change in output when Alice is part of the team.
- **Bob's SHAP Value:** The average change in output when Bob is part of the team.
- **Charlie's SHAP Value:** The average change in output when Charlie is part of the team.

![Shapley](shap.png)

In [ ]:
import shap
# Create explanations

In [ ]:
X_test_target.head()

SHAP values are a measure used to explain the output of machine learning models. They tell us how much each feature in the model contributes to a particular prediction, compared to a baseline prediction.

#### Positive vs. Negative SHAP Values

- **Positive SHAP Values:** When a feature has a positive SHAP value, it means that the presence (or value) of that feature pushes the model's output higher than the baseline.
  - **Example:** In our lateral spreading prediction the PGA (a feature) gives a positive SHAP value, it means that higher PGA are generally associated with higher chance of lateral spreading.

- **Negative SHAP Values:** Conversely, when a feature has a negative SHAP value, it means that the presence (or value) of that feature pushes the model's output lower than the baseline.
  - **Example:** Still considering lateral spreading, if water table being too low (a feature) gives a negative SHAP value, it suggests that such sites are generally predicted to have less chance of lateral spreading.


The term $E(f(x))$ represents the expected value or average prediction of the model over the entire dataset. Think of it as the "baseline prediction." When you see $E(f(x))$ , it's referring to what the model would predict on average, without considering any specific feature values.

- **Example:** In the context of predicting house prices, $E(f(x))$  might be the average chance of lateral spreading the model predicts when it doesn't consider any specific features of the sites. It's the starting point before we account for unique factors like having GWT or a high PGA.

#### Positive prediction
In this example, we show a site that is predicted to have positive lateral spreading (failure). Here, the high slope anlge (0.835%) has the most influence in the positive prediction of lateral spreading, followed by a shallow ground water depth of 1.8 m and its close proximity to the river (L = 0.74 km).

#### Negative prediction

In this example, we show XGB predicting a site that has no chance of lateral spreading. The distance to the river (L ~ 2 km) seems to be the primary reason.

### Global explanation

We also observe global SHAP values and see how different features influence the results. High feature values are shown in red and Low feature values are shown in blue. For e.g., shorter distance to the river (blue points for $L$) are associated with high chance of lateral spreading. Similarly, deeper ground water depth (red GWD) means lower chances of lateral spreading.

However, we do see that high PGA, shows less chance of lateral spreading, contradictory to the engineering understanding that high acceleration will cause more displacements and failures.